In [6]:
import torch
import torch.nn as nn


In [5]:
# visualization
!pip install torchview torchinfo
import torchview
import torchinfo
from torchview import draw_graph
from IPython.display import Image


### Load pretrained model summary

In [7]:
import torchvision
from torchvision.models import resnet34,resnet50,resnet18,resnet101,resnet152,ResNet50_Weights
torchvision_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 83.2MB/s]


In [10]:

# from torchsummary import summary
# summary(torchvision_model, (3, 224, 224))

from torchinfo import summary
summary(torchvision_model, (3,224,224), batch_dim=0, verbose=1)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

In [13]:
# model_graph = draw_graph(torchvision_model, input_size=(1,3,32,32), expand_nested=True)
# model_graph.visual_graph

### Resnet Architecture

In [20]:
# Define model parameters for different resnet versions

# resnetX = (Num_of_channels, repetition, Bottleneck_expansion, Bottleneck_layer)
model_parameters = {}
model_parameters['resnet_18'] = ([64,128,256,512], [2,2,2,2], 1, False)
model_parameters['resnet_34'] = ([64,128,256,512], [3,4,6,3], 1, False)
model_parameters['resnet_50'] = ([64,128,256,512], [3,4,6,3], 4, True)
model_parameters['resnet_101'] = ([64,128,256,512],[3,4,23,3], 4, True)
model_parameters['resnet_152'] = ([64,128,256,512], [3,8,36,3], 4, True)

In [22]:
model_parameters['resnet_50']   # ([64, 128, 256, 512], [3, 4, 6, 3], 4, True)

# - [64,128,256,512] --> channels in each intermediate block
# - [3,4,6,3]        --> repeatition for Bottlenecks in each block
# - 4                --> expansion_factor. Note that 64 turns to 256, 128 to 512. All the resnet layers use the same expansion factor.
# - True             --> create Bottleneck layer status. True only for ResNet 50 model and above

([64, 128, 256, 512], [3, 4, 6, 3], 4, True)

#### Bottleneck Blocks Vs BasicBlocks

- For resnet-18/34, BasicBlocks were used instead of bottleneck.
- Bottleneck is used to reduce the computation cost for layers-50,101 & 152.


A skip connection is established from the input of a residual block to the point just prior to the final ReLU activation, where the fusion of feature maps occurs. This skip connection can take on one of two forms:
- it can either maintain the identity of the input -> **Identity** (feature map size of input and output are same)
- undergo a projection -> **Projected** (feature map size of input and output are different)

In [24]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, expansion, is_Bottleneck, stride):
        """
        Function with bottleneck block(resnet50/101/152) and basic block(resnet18/34)

        Creates a Bottleneck with conv 1x1->3x3->1x1 layers.

        Note:
          1. Addition of feature maps occur at just before the final ReLU with the input feature maps
          2. if input size is different from output, select projected mapping or else identity mapping.
          3. if is_Bottleneck=False (3x3->3x3) are used else (1x1->3x3->1x1). Bottleneck is required for resnet-50/101/152
        Args:
            in_channels (int) : input channels to the Bottleneck
            out_channels (int) : number of channels to 3x3 conv
            expansion (int) : factor by which the input #channels are increased
            stride (int) : stride applied in the 3x3 conv. 2 for first Bottleneck of the block and 1 for remaining

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """

        super(Bottleneck,self).__init__()

        self.expansion = expansion
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.is_Bottleneck = is_Bottleneck
        self.relu = nn.ReLU()

        # Define skip connections(identity/projected) based on input/output shape
        if self.in_channels==self.out_channels*self.expansion:
            self.identity = True
        else:
            self.identity = False
            projection_layer = []
            projection_layer.append(nn.Conv2d(in_channels=self.in_channels,
                                              out_channels=self.out_channels*self.expansion,
                                              kernel_size=1, stride=stride, padding=0))
            projection_layer.append(nn.BatchNorm2d(self.out_channels*self.expansion))
            self.projection = nn.Sequential(*projection_layer)


        # Implement basic blocks for Resnet18/34 and bottleneck blocks for Resnet50/101/152
        if self.is_Bottleneck:
            # bottleneck
            # 1x1
            self.conv1_1x1 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=1,
                                       stride=1, padding=0)
            self.batchnorm1 = nn.BatchNorm2d(self.out_channels)

            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.out_channels, out_channels=self.out_channels, kernel_size=3,
                                       stride=stride, padding=1)
            self.batchnorm2 = nn.BatchNorm2d(self.out_channels)

            # 1x1
            self.conv3_1x1 = nn.Conv2d(in_channels=self.out_channels, out_channels=self.out_channels*self.expansion, kernel_size=1,
                                       stride=1, padding=0)
            self.batchnorm3 = nn.BatchNorm2d( self.out_channels*self.expansion )

        else:
            # basicblock
            # 3x3
            self.conv1_3x3 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=3,
                                       stride=stride, padding=1)
            self.batchnorm1 = nn.BatchNorm2d(self.out_channels)

            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.out_channels, out_channels=self.out_channels, kernel_size=3,
                                       stride=1, padding=1)
            self.batchnorm2 = nn.BatchNorm2d(self.out_channels)

    def forward(self,x):
        # input stored to be added before the final relu
        shortcut = x

        if self.is_Bottleneck:
            # conv1x1->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_1x1(x)))

            # conv3x3->BN->relu
            x = self.relu(self.batchnorm2(self.conv2_3x3(x)))

            # conv1x1->BN
            x = self.batchnorm3(self.conv3_1x1(x))

        else:
            # conv3x3->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_3x3(x)))

            # conv3x3->BN
            x = self.batchnorm2(self.conv2_3x3(x))


        # identity or projected mapping
        if self.identity:
            x += shortcut
        else:
            x += self.projection(shortcut)

        # final relu
        x = self.relu(x)

        return x


In [29]:
# Bottleneck(64*4,64,4,stride=1)

def test_Bottleneck():
    x = torch.randn(1,64,112,112)
    model = Bottleneck(64,64,4,True,2)
    print(model(x).shape)
    del model

test_Bottleneck()

torch.Size([1, 256, 56, 56])


#### Resnet block

In [32]:
class ResNet(nn.Module):

    def __init__(self, resnet_variant, in_channels, num_classes):
        """
        Creates the ResNet architecture based on the provided variant. 18/34/50/101 etc.
        Based on the input parameters, define the channels list, repeatition list along with expansion factor(4) and stride(3/1)
        using _make_blocks method, create a sequence of multiple Bottlenecks
        Average Pool at the end before the FC layer

        Args:
            resnet_variant (list) : eg. [[64,128,256,512],[3,4,6,3],4,True]
            in_channels (int) : image channels (3)
            num_classes (int) : output #classes

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """
        super(ResNet,self).__init__()
        self.channels_list = resnet_variant[0]
        self.repeatition_list = resnet_variant[1]
        self.expansion = resnet_variant[2]
        self.is_Bottleneck = resnet_variant[3] # basic block or bottleneck block

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.block1 = self._make_blocks( 64 , self.channels_list[0], self.repeatition_list[0], self.expansion, self.is_Bottleneck,
                                        stride=1 )
        self.block2 = self._make_blocks( self.channels_list[0]*self.expansion , self.channels_list[1], self.repeatition_list[1],
                                        self.expansion, self.is_Bottleneck, stride=2 )
        self.block3 = self._make_blocks( self.channels_list[1]*self.expansion , self.channels_list[2], self.repeatition_list[2],
                                        self.expansion, self.is_Bottleneck, stride=2 )
        self.block4 = self._make_blocks( self.channels_list[2]*self.expansion , self.channels_list[3], self.repeatition_list[3],
                                        self.expansion, self.is_Bottleneck, stride=2 )

        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear( self.channels_list[3]*self.expansion , num_classes)



    def forward(self,x):
        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = self.average_pool(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)

        return x

    def _make_blocks(self, in_channels, intermediate_channels, num_repeat, expansion, is_Bottleneck, stride):

        """
        Args:
            in_channels : #channels of the Bottleneck input
            intermediate_channels : #channels of the 3x3 in the Bottleneck
            num_repeat : #Bottlenecks in the block
            expansion : factor by which intermediate_channels are multiplied to create the output channels
            is_Bottleneck : status if Bottleneck in required
            stride : stride to be used in the first Bottleneck conv 3x3

        Attributes:
            Sequence of Bottleneck layers

        """
        layers = []

        layers.append(Bottleneck(in_channels, intermediate_channels, expansion, is_Bottleneck, stride=stride))
        for num in range(1,num_repeat):
            layers.append(Bottleneck(intermediate_channels*expansion,intermediate_channels,expansion,is_Bottleneck,stride=1))

        return nn.Sequential(*layers)



In [34]:
def test_ResNet(params):
    model = ResNet( params , in_channels=3, num_classes=1000)
    x = torch.randn(1,3,224,224)
    output = model(x)
    print(output.shape)
    return model

architecture = 'resnet_50'
model = test_ResNet(model_parameters[architecture])

torch.Size([1, 1000])


In [35]:
from torchinfo import summary
summary(model, (3,224,224), batch_dim=0, verbose=1)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,160
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,928
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,472
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,160
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,928
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

### References

- [Writing ResNet from Scratch in PyTorch](https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/)

- [Resnet Implementation In Pytorch ](https://medium.com/@karuneshu21/how-to-resnet-in-pytorch-9acb01f36cf5)

- [Aladdinpersson pytorch resnet](https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/CNN_architectures/pytorch_resnet.py)

- [A Detailed Introduction to ResNet and Its Implementation in PyTorch](https://medium.com/@freshtechyy/a-detailed-introduction-to-resnet-and-its-implementation-in-pytorch-744b13c8074a)